<a href="https://colab.research.google.com/github/fberetta8/ML_in_Finance_Spring_2025_ETH/blob/main/Notebook8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Pricing in SABR model
The **SABR model** (Stochastic Alpha, Beta, Rho) is a stochastic volatility model commonly used in the pricing of derivatives, especially interest rate options and swaptions.

It models the forward price $F_t$ and its volatility $\sigma_t$ using the following system of SDEs:

$$
\begin{aligned}
dF_t &= \sigma_t F_t^{\beta} dW_t, \\
d\sigma_t &= \nu \sigma_t dB_t, \\
\end{aligned}
$$

with:

- $\beta \in [0, 1]$: controls the dependence of volatility on the price (elasticity).
- $\nu > 0$: volatility of volatility.
- $\rho \in [-1, 1]$: correlation between the two Brownian motions $W_t$ and $B_t$.

The initial volatility is denoted $\alpha = \sigma_0$. We aim to learn the map

$$
(\alpha, \beta, \rho, \nu) \rightarrow \text{price}
$$


# Hagan Approximation

Since the SABR model does **not** admit a closed-form solution for option prices, practitioners typically use the **Hagan et al. approximation** to compute **implied volatilities** under SABR. This implied volatility can then be inserted into the **Black-Scholes formula** to compute prices. In this notebook, we rely on this approximation to simulate data and train a machine learning model to learn the price function.

In [ ]:
# Libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd

In [ ]:
# Implied Volatility Formula
def sabr_implied_vol(F, K, T, alpha, beta, rho, nu):
    if F == K:
        term1 = (1 + ((1 - beta)**2 / 24) * (alpha**2 / F**(2 - 2*beta))
                 + (rho * beta * nu * alpha) / (4 * F**(1 - beta))
                 + (nu**2 * (2 - 3 * rho**2)) / 24) * T
        return alpha / (F**(1 - beta)) * (1 + term1)

    log_FK = np.log(F / K)
    z = (nu / alpha) * (F * K)**((1 - beta)/2) * log_FK
    x_z = np.log((np.sqrt(1 - 2 * rho * z + z**2) + z - rho) / (1 - rho))
    FK_beta = (F * K)**((1 - beta)/2)

    term1 = (1 + ((1 - beta)**2 / 24) * (log_FK)**2
             + ((1 - beta)**4 / 1920) * (log_FK)**4)
    term2 = (1 + ((1 - beta)**2 / 24) * (alpha**2 / FK_beta**2)
             + (rho * beta * nu * alpha) / (4 * FK_beta)
             + (nu**2 * (2 - 3 * rho**2)) / 24) * T

    return (alpha / (FK_beta * term1)) * (z / x_z) * (1 + term2)

In [ ]:
# Given the implied volatility, we compute the price of a European
# call option using the Black-Scholes formula.

def black_scholes_call(F, K, T, sigma):
    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return F * norm.cdf(d1) - K * norm.cdf(d2)

In [ ]:
# Generate data

def generate_data(n_samples=10000, F=100.0, K=100.0, T=1.0):
    alphas = np.random.uniform(0.05, 0.5, n_samples)
    betas = np.random.uniform(0.0, 1.0, n_samples)
    rhos = np.random.uniform(-0.99, 0.99, n_samples)
    nus = np.random.uniform(0.1, 1.0, n_samples)

    X = np.stack([alphas, betas, rhos, nus], axis=1)
    y = []

    for alpha, beta, rho, nu in X:
        sigma = sabr_implied_vol(F, K, T, alpha, beta, rho, nu)
        price = black_scholes_call(F, K, T, sigma)
        y.append(price)

    return X, np.array(y)

In [ ]:
# Preprocess data and divide train/test set

X, y = generate_data()
y_log = np.log1p(y) # We apply log-transform, it helps the fit
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train_log, y_test_log = train_test_split(
    X_scaled, y_log, test_size=0.2, random_state=42
)

X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train_log, dtype=torch.float32).view(-1, 1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test_log, dtype=torch.float32).view(-1, 1)

In [ ]:
# TODO: Build a model based on a feed-forward NN. Train it on the train set and
#       make predictions over the test set.

In [ ]:
# TODO: Visualize the predicted prices vs the one computed via Hagan approx
#       (assumed to be the "real" prices), over the test set.